# Test for earth-rovers-sdk
For a better understanding of how this SDK work.

## 1. asyncio

In [7]:
import asyncio
async def hello():
    print("Hello world!")
    await asyncio.sleep(1)
    print("Hello again!")
await hello()
# asyncio.run(hello()) cannot be called when another asyncio loop is running, but the jupyter is already running an event loop

Hello world!
Hello again!


This time we practice to get the sina/sohu and 163 website using `asyncio`

In [ ]:
async def wget(host):
    print(f"wget {host}...")
    reader, writter = await asyncio.open_connection(host, 80)
    header = f"GET / HTTP/1.0\r\nHost: {host}\r\n\r\n"
    writter.write(header.encode('utf-8'))
    await writter.drain()

    while True:
        line = await reader.readline()
        if line == b"\r\n":
            break
        print("%s header > %s" % (host, line.decode('utf-8').rstrip()))
    writter.close()
    await writter.wait_closed()
    print(f"Done {host}.")

async def main():
    await asyncio.gather(
        wget("www.sina.com.cn"),
        wget("www.sohu.com"),
        wget("www.163.com")
    )

await main()

## 2. JSON
In the [rtm_client](./rtm_client.py), what is the `RTM_TOKEN` about?
The json.dumps method, which come in with a message and seperators, is explained as follows:
1. Serialization: It converts a Python object (like a dictionary or list) into its equivalent JSON string representation
2. `message` argument: This is the Python object that needs to be serialized
3. `separators` argument: This argumet controls the characters used to separate items in arrays and key-value pairs in objects within the JSON output. It expects a two-tuple: `(item_separator, key_separator)`
Effect of `swparators=(',', ':')`:
* Copact Output: By specifying `separators=(',', ':')`, you are instructing `json.dumps` to use a comma (`,`) as the item separator in arrays and a colon (`:`) as the key-value separator in objects, without any additional whitespace.
* Reduced Size: This results in the most compat JSON representation possible, as it eliminates all unnecessary spaces typically added for readability in default or pretty-printed JSON. This is particularly useful when minimizing file size or network bandwidth is a priority.

Here's an example:

In [9]:
import json

data = {
    "name": "Yixun Hu",
    "age": 22,
    "university": "Princeton"
}

# Using default separators (often includes spaces)
default_json = json.dumps(data)
print("Default JSON:", default_json)

# Using compact separators
compact_json = json.dumps(data, separators=(',', ':'))
print("Compact JSON:", compact_json)

Default JSON: {"name": "Yixun Hu", "age": 22, "university": "Princeton"}
Compact JSON: {"name":"Yixun Hu","age":22,"university":"Princeton"}


## 3. Package of requests: `requests.post()`

The Python `requests.post()` metho, when used with `url`, `headers`, and `json` parameters, sends a POST request iwth a JSON payload to a specified URL.
Explanation of parameters:
1. `url`: This is a string representing the target URL to which the POST request will be sent. 
    1. What is the POST?
        - The `post()` method is used when you want to send some data to the server.
    2. Syntax: `requests.post(url, data={key:value}, json={key: value}, args)`
        - The *args* means zero or more of the *named* arguments in the parameter table below. Example is `requests.post(url, data=myjob, timeout=2.50)`
2. `headers`: This parameter allows you to include custom HTTP headers in your request. 
3. `json=payload`: This parameter is used to send JSON data in the request body. `payload` should be a Python dictionary or list that will be serialized into a JSON string by the `requests` library. The `requests` library will automatically sets the `Content-Type` header to `application/json`.
4. `response`: This variable will store the `Response` object returned by the `requests.post()` call.

In [ ]:
# This is the example
import requests
url = "https://www.w3schools.com/python/demopage.php"
url = "https://github.com/frodobots-org/earth-rovers-sdk?tab=readme-ov-file"
myobj = {
    "somekey": "somevalue"
}
x = requests.post(url, json=myobj)
print(x.text)

## 4. Decorators
* `app.get("/screenshot")`: This is a decorator that associates the function it decorates with the specified URL path (`/screenshot`) and the HTTP GET method.
* `app`: This usually refers to an instance of a web application framework (e.g., `Flask(__name__)` or `FastAPI()`)
* `/screenshot`: This is the specific URL path that the endpoint will respons to

# Get the images

In [55]:
! curl --location 'http://localhost:8000/v2/screenshot'

{"front_frame":"...","timestamp":1758674800.309059}

In [167]:
import base64
import os
import requests

response = requests.get('http://localhost:8000/v2/screenshot')
print(response.status_code)
data = response.json()
print(data['timestamp'])
front_base64 = data['front_frame']

with open('front_frame.jpeg', 'wb') as f:
    f.write(base64.b64decode(front_base64))


200
1758676112.474365


In [206]:
!curl --location 'http://localhost:8000/data'

{"signature":"540c8ac9641ca9e7b92d45f530283c369c2709d7c6b09a0a3029c833056a97cc","battery":77,"signal_level":3,"orientation":26,"lamp":0,"speed":0,"gps_signal":0,"latitude":1000,"longitude":1000,"vibration":0,"hdop":0,"altitude":0,"fix_quality":0,"distance_to_rtk_station":-1,"gps_ttff":0,"power":3,"current":114,"voltage":27,"network_state":2,"accels":[[-0.009,0.004,1.007,1758672032.245],[-0.012,0.001,1.015,1758672032.268],[-0.005,-0.001,1.014,1758672032.288],[-0.008,0.004,1.01,1758672032.305],[-0.011,0.004,1.007,1758672032.328]],"gyros":[[-0.061,0.122,0.305,1758672032.245],[-0.122,0,0.244,1758672032.268],[0,-0.122,0.244,1758672032.288],[-0.061,0.122,0.244,1758672032.305],[-0.122,0.061,0.305,1758672032.328]],"mags":[[1283,-2574,-1414,1758672032.328]],"rpms":[[0,0,0,0,1758672032.245],[0,0,0,0,1758672032.268],[0,0,0,0,1758672032.288],[0,0,0,0,1758672032.305],[0,0,0,0,1758672032.328]],"timestamp":"1758672037.576000"}

In [207]:
!curl --location 'http://localhost:8000/control' --header 'Content-Type: application/json' --data '{ "command": { "linear": 1, "angular": 1 }}'

{"message":"Command sent successfully"}

In [208]:
!curl --location 'http://localhost:8000/v2/front'

{"front_frame":"...","timestamp":1758686442.220031}

In [188]:
import base64
import os
import requests

response = requests.get('http://localhost:8000/screenshot?view_types=front')
print(response.status_code)
data = response.json()
print(data.keys())
front_base64 = data['front_frame']

with open('front_frame.jpeg', 'wb') as f:
    # print(len(front_base64))
    # print(len(base64.b64decode(front_base64)))
    # print(type(front_base64), type(base64.b64decode(front_base64)))
    f.write(base64.b64decode(front_base64))


200
dict_keys(['front_frame', 'timestamp'])


In [ ]:
import time
import requests
import base64
while True:
    response = requests.get('http://localhost:8000/screenshot?view_types=front')
    data = requests.get('http://localhost:8000/data').json()
    timestamp_img = data['timestamp']
    image_data = response.json()
    front_base64 = image_data['front_frame']
    if not os.path.exists('./data'):
        os.makedirs('./data')
    with open(f'./data/front_frame_{timestamp_img}.jpeg', 'wb') as f:
        f.write(base64.b64decode(front_base64))
    time.sleep(1)

In [ ]:
data = requests.get('http://localhost:8000/data').json()
print(data['timestamp'])
from datetime import datetime, timedelta, timezone
from math import floor

# Example Unix timestamp (in seconds)
unix_timestamp = floor(float(data['timestamp']))

# Convert the Unix timestamp to a datetime object
dt_object = datetime.fromtimestamp(unix_timestamp)

# Print the datetime object
print(dt_object)

# Convert to a timezone-aware datetime object (e.g., UTC)
dt_utc_aware = datetime.fromtimestamp(unix_timestamp, tz=timezone.utc)
print(dt_utc_aware)

# Convert to a timezone-aware datetime object with a custom offset (e.g., EST)
dt_est_aware = datetime.fromtimestamp(unix_timestamp, tz=timezone(timedelta(hours=+8), 'EST'))
print(dt_est_aware)

In [114]:
!date -Ins +"%Y-%m-%d %H:%M:%S"

date: multiple output formats specified


# B64 images to video

In [ ]:
import numpy as np

# Create a byte string
data_bytes = b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'

# Interpret the byte string as an array of 32-bit unsigned integers
arr = np.frombuffer(data_bytes, dtype=np.uint8)
print(arr)

# Interpret a portion of the byte string with a different dtype
partial_arr = np.frombuffer(data_bytes, dtype=np.uint16, offset=4, count=2)
print(partial_arr)

This is the code that can capture the image data from the localhost:8000

In [1]:
import requests
import time
import os
import base64
import numpy as np
import cv2
idx = 1
def b64_to_bgr(b64_str: str) -> np.ndarray:
    """Decode base64 JPEG/PNG string to BGR image (OpenCV)."""
    try:
        buf = base64.b64decode(b64_str)
        print(buf)
        print(type(buf), len(buf))
        arr = np.frombuffer(buf, dtype=np.uint8)
        print(arr)
        print(f"arr type: {arr.dtype}, shape: {arr.shape}")
        img = cv2.imdecode(arr, cv2.IMREAD_COLOR)  # BGR
        return img
    except Exception:
        return None
while True:
    try:
        response = requests.get('http://localhost:8000/v2/front')
        img_data = response.json()
    except Exception as e:
        print(f"Error fetching screenshot: {e}")
        break
    timestamp_img = img_data['timestamp']
    front_base64 = img_data['front_frame']
    img = b64_to_bgr(front_base64)
    if not os.path.exists(f'./data_{idx}'):
        os.makedirs(f'./data_{idx}')
    if img is not None:
        # print(img.shape)
        cv2.imwrite(f'./data_{idx}/front_frame_{timestamp_img}.jpeg', img)
    time.sleep(0.03333333333333333)  # ~30 FPS
print("Exited loop.")

KeyboardInterrupt: 

In [2]:
# make video

import re, glob, os
import cv2
import numpy as np

DATA_DIR = './data_1'
OUTPUT_VIDEO = 'worldrover_front_0923.mp4'
image_files = glob.glob(os.path.join(DATA_DIR, 'front_frame_*.jpeg'))

In [ ]:
image_files

In [3]:
pattern = re.compile(r"front_frame_(\d+\.\d+)\.jpe?g$")
pairs = []
for f in image_files:
    m = pattern.search(f)
    if m:
        ts = float(m.group(1))  
        pairs.append((ts, f))
pairs.sort(key=lambda x: x[0])  # 按时间从早到晚
sorted_image_files = [f for ts, f in pairs]

In [4]:
def pad_to_even(img):
    h, w = img.shape[:2]
    H = (h + 1) // 2 * 2
    W = (w + 1) // 2 * 2
    if (H, W) == (h, w):
        return img
    return cv2.copyMakeBorder(img, 0, H - h, 0, W - w, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))

In [5]:
first = cv2.imread(image_files[0])
if first is None:
    print("Failed to read first frame.")
first = pad_to_even(first)
H, W = first.shape[:2]
H, W

(576, 1024)

In [7]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw30 = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, 30, (W, H))

In [8]:
for i, fp in enumerate(sorted_image_files, 1):
    img = cv2.imread(fp)
    if img is None:
        print(f"Skip unreadable frame: {fp}")
        continue
    # 统一尺寸（若与你的采集流程保证了尺寸一致，可去掉这两行）
    if img.shape[:2] != (H, W):
        img = cv2.resize(img, (W, H), interpolation=cv2.INTER_AREA)
    img = pad_to_even(img)  # 确保偶数宽高（H.264 友好）

    vw30.write(img)

    if i % 100 == 0:
        print(f"Wrote {i} frames...")

vw30.release()
print(f"Done! Saved to {OUTPUT_VIDEO}")

Wrote 100 frames...
Wrote 200 frames...
Wrote 300 frames...
Wrote 400 frames...
Wrote 500 frames...
Wrote 600 frames...
Wrote 700 frames...
Wrote 800 frames...
Wrote 900 frames...
Wrote 1000 frames...
Wrote 1100 frames...
Wrote 1200 frames...
Wrote 1300 frames...
Wrote 1400 frames...
Wrote 1500 frames...
Wrote 1600 frames...
Wrote 1700 frames...
Wrote 1800 frames...
Wrote 1900 frames...
Wrote 2000 frames...
Wrote 2100 frames...
Wrote 2200 frames...
Wrote 2300 frames...
Wrote 2400 frames...
Wrote 2500 frames...
Wrote 2600 frames...
Wrote 2700 frames...
Wrote 2800 frames...
Done! Saved to worldrover_front_0923.mp4
